In [1]:
# pRNN wavefunction ansazt with GRU layer + Dense softmax
# Samples homogeneous monomials in Nc^2 and fixed degree 
# Needs to be fixed by broadcasting

import tensorflow as tf
import numpy as np


class RNNWavefunction(tf.keras.Model):
    def __init__(self, system_size, units=20, input_dim=3, output_dim=3, seed=211):
        """
        system_size: int, number of timesteps or system size (= number of monomial variables)
        units: int, number of units in the GRU layer
        input_dim: int, number of input features (= monomial homogeneous degree = charge+1)
        output_dim: int, number of output features (= monomial homogeneous degree = charge+1)
        seed: int, the random seed for reproducibility
        """
        super(RNNWavefunction, self).__init__()

        # Set random seeds for reproducibility
        np.random.seed(seed)
        tf.random.set_seed(seed)

        self.system_size = system_size
        self.input_dim = input_dim
        self.output_dim = output_dim
        
        # Define the GRU layer (one GRU layer with specified units)
        self.gru = tf.keras.layers.GRU(units=units, return_sequences=True, return_state=True)
        
        # Final Dense layer with Softmax output (probabilities, not logits!)
        self.dense = tf.keras.layers.Dense(output_dim, activation="softmax")
        
    def call(self, inputs, hidden_state=None, training=False):
        """
        Forward pass through the network with fixed hidden state.
        """
        if hidden_state is None:
            hidden_state = tf.zeros((inputs.shape[0], self.gru.units))  # Fixed hidden state

        x, hidden_state = self.gru(inputs, initial_state=hidden_state, training=training)  # GRU layer
        x = self.dense(x)  # Apply Dense layer

        return x, hidden_state

    def sample(self, numsamples):

        """
        Generate samples from the probability distribution parameterized by the RNN.
        numsamples: int, number of samples to generate
        """
        samples = []  # List to store the generated sequence
        inputs = tf.zeros((numsamples, 1, self.input_dim), dtype=tf.float32)  # Initial input (zero vector)
        hidden_state = None  # No initial hidden state

        for t in range(self.system_size-1): # Sampling cycle over system_size =  number of variables
            output, hidden_state = self.call(inputs, hidden_state=hidden_state)  # Forward pass through the model

            # Get probabilities for the last generated timestep
            softout= output[:, -1, :]  # Shape: [numsamples, output_dim]
            #print("Sample is now = "+str(samples))
            # Projection of softmax probabilities imposing charge conservation
            softout_t = np.copy(softout)
            #print("Starting with softout_t = "+str(softout))
            thetavec = [np.heaviside(self.input_dim-1-np.sum(np.array(samples),axis=0)-i,1) for i in range(softout_t.shape[1])]
            #print("Apply projection thetavec = "+str(thetavec))
            softout_t = np.array([softout_t[:,i]*thetavec[i] for i in range(softout_t.shape[1])])

            #norms = np.sum(softout_t,axis=0)
            #print("Normalize = "+str(softout_t))
            #print("by norm vec = "+str(norms))
            softout_t = np.transpose(softout_t)
            #print(softout_t / np.sum(softout_t, axis=1, keepdims=True))
            

            # Sample from categorical distribution
        
            sampled_t = tf.random.categorical(tf.math.log(softout_t), num_samples=1)  # Shape: [numsamples, 1]
            sampled_t = tf.squeeze(sampled_t, axis=-1)  # Shape: [numsamples]
        
            # Append sampled values to the list
            samples.append(sampled_t)
        
            # Convert sampled values to one-hot encoding for the next input
            inputs = tf.one_hot(sampled_t, depth=self.output_dim, dtype=tf.float32)
            inputs = tf.expand_dims(inputs, axis=1)  # Add time-step dimension
            
        samples = tf.stack(samples, axis=1)

        J = tf.constant((self.input_dim-1)*np.ones(samples.shape[0])-np.sum(samples,axis=1))
        J = np.transpose(tf.cast(tf.expand_dims(J, axis=0), dtype = tf.int64))
        #print(samples)
        #print(J)
        samples = tf.concat([samples, J], axis=1)

        return samples

    def log_probability(self, samples):
        """
        Calculate log-probabilities of the given samples.
        samples: Tensor, shape (numsamples, system_size), the sampled wavefunction
        """
        # Convert samples to one-hot encoding
        one_hot_samples = tf.one_hot(samples, depth=self.output_dim, dtype=tf.float32)

        inputs = one_hot_samples  # Shape: [numsamples, system_size, output_dim]
    
        # Ensure evaluation mode (training=False)
        probs, _ = self.call(inputs, training=False)  # Forward pass through the model with training=False
        
        # Compute log probabilities (log(p(x)))
        log_probs = tf.reduce_sum(tf.math.log(tf.reduce_sum(tf.multiply(probs, one_hot_samples), axis=-1)), axis=-1)

        return log_probs

In [120]:
# Parameters
Nc = 2  
system_size = Nc*Nc # Number of timesteps
charge = 10 # Polynomial Degree
input_dim = charge+1  # Number of input features = total charge
output_dim = charge+1   # Number of output classes = total charge
units = 1     # Number of
# GRU units
numsamples = 10  # Number of samples to generate
seed = 221      # Random seed for reproducibility

# Instantiate the RNNWavefunction model
model = RNNWavefunction(system_size, units, input_dim, output_dim, seed=np.random.randint(1,100))

# Example: Sampling
samples = model.sample(numsamples)
print(samples)
#print(f"Generated Samples:\n{samples.numpy()}")


tf.Tensor(
[[ 4  5  1  0]
 [ 5  4  0  1]
 [ 1  3  0  6]
 [ 7  2  1  0]
 [ 3  4  3  0]
 [ 3  5  1  1]
 [10  0  0  0]
 [ 2  5  1  2]
 [ 4  4  2  0]
 [ 5  0  1  4]], shape=(10, 4), dtype=int64)


In [117]:
def symPoly_local_energies(a,b,Nc, samples, model):
    """ Local energies of a system of Nc**2 interacting bosonic oscillators.
    Returns: The local energies that correspond to the "samples"
    Inputs:
    - a,b: define the Gauge generator
    - Nc: rank of the matrix group
    - samples: (numsamples, Nc**2)
    - model: The RNN wavefunction model instance
    """

    numsamples = samples.shape[0]  # Extracts number of samples

    local_energies = np.zeros((numsamples), dtype=np.float64)  # Initialize local energy to zero for each sample
    
    energy_samples= np.zeros((2*Nc,numsamples), dtype=np.float64) # Array of energies to zero for each sample
    #print(energy_samples)
    queue_samples= np.zeros((1+2*Nc,numsamples,Nc*Nc), dtype=np.float64)  # Array of vector states to zero for each sample
    log_probs= np.zeros((1+2*Nc)*numsamples, dtype=np.float64) # Array of log probs for each vector state for each sample
    
    #print(samples.shape)
    #print(queue_samples[0].shape)
    queue_samples[0]=samples

    # Evaluation of local energy

    for c in range(Nc):  # +1 terms
        samplesT = np.copy(samples)
        
        energy_samples[c,:][samples[:, (a-1)*Nc+c] > 0]= 1
        energy_samples[c,:][samples[:, (a-1)*Nc+c] == 0]= 0

        #print(energy_samples[c,:])
        
        samplesT[:, (a-1)*Nc+c] -= 1
        samplesT[:, (b-1)*Nc+c] += 1
        
        queue_samples[1+c] = samplesT

    for c in range(Nc):  # -1 terms
        samplesT = np.copy(samples)

        energy_samples[c+Nc,:][samples[:, c*Nc+(b-1)] > 0]= -1
        energy_samples[c+Nc,:][samples[:, c*Nc+(b-1)] == 0]= 0
        
        #print(energy_samples[c+Nc,:])
        
        samplesT[:, c*Nc+(a-1)] += 1
        samplesT[:, c*Nc+(b-1)] -= 1
        
        queue_samples[1+Nc+c] = samplesT
 
    # Evaluate log probability of samples and of flipped sample vectors, according to pRNN amplitudes
    
    queue_samples_reshaped = np.reshape(queue_samples, [(1+2*Nc) * numsamples, Nc*Nc])
    len_sigmas = queue_samples_reshaped.shape[0]

    steps = np.ceil(len_sigmas / 25000)  # Maximum of 25000 configurations in batch size for memory reasons

    for i in range(int(steps)):
        cut = slice((i * len_sigmas) // int(steps), ((i + 1) * len_sigmas) // int(steps))
        log_probs[cut] = model.log_probability(queue_samples_reshaped[cut]) # Computes log probabilities slice-by-slice

    log_probs_reshaped = np.reshape(log_probs,[1+2*Nc,numsamples]) # Reshape log_probs putting in line all log probs related to a given sample
    amplitudes_ratio = np.exp(0.5 * log_probs_reshaped[1:, :] - 0.5 * log_probs_reshaped[0, :]) #Ratio of wavefunction amplitudes for each flipped over unflipped sample

    local_energies += np.sum(energy_samples*amplitudes_ratio,axis=0) # Adds local energy from non-diagonal terms 

    return local_energies

Normalize = [[0.33333334]
 [0.33333334]
 [0.33333334]]
by norm vec = [1.]
[[0.33333334 0.33333334 0.33333334]]
Normalize = [[0.30364725]
 [0.35484487]
 [0.        ]]
by norm vec = [0.6584921]
[[0.46112514 0.5388749  0.        ]]
Normalize = [[0.25570488]
 [0.39163306]
 [0.        ]]
by norm vec = [0.6473379]
[[0.3950099 0.6049901 0.       ]]
tf.Tensor([[2 0 0]], shape=(1, 3), dtype=int64)
